<a href="https://colab.research.google.com/github/LionelSh/Jedha-GetAround-B5/blob/main/Getaround_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Imports

In [ ]:
!pip install -U scikit-learn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 48.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay, RocCurveDisplay
import joblib
import pickle


In [ ]:
df = pd.read_csv('/content/get_around_pricing_project.csv')

# 2) EDA

In [ ]:
df.head(10)

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183
5,5,Citroën,152352,225,petrol,black,convertible,True,True,False,False,True,True,True,131
6,6,Citroën,205219,145,diesel,grey,convertible,True,True,False,False,True,True,True,111
7,7,Citroën,115560,105,petrol,white,convertible,True,True,False,False,False,True,True,78
8,8,Peugeot,123886,125,petrol,black,convertible,True,False,False,False,False,True,True,79
9,9,Citroën,139541,135,diesel,white,convertible,False,False,False,False,True,False,True,132


In [ ]:
j = df.loc[8]
print(j)

Unnamed: 0                             8
model_key                        Peugeot
mileage                           123886
engine_power                         125
fuel                              petrol
paint_color                        black
car_type                     convertible
private_parking_available           True
has_gps                            False
has_air_conditioning               False
automatic_car                      False
has_getaround_connect              False
has_speed_regulator                 True
winter_tires                        True
rental_price_per_day                  79
Name: 8, dtype: object


In [ ]:
df.columns

Index(['Unnamed: 0', 'model_key', 'mileage', 'engine_power', 'fuel',
       'paint_color', 'car_type', 'private_parking_available', 'has_gps',
       'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
       'has_speed_regulator', 'winter_tires', 'rental_price_per_day'],
      dtype='object')

In [ ]:
df.describe(include='all')

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843.000000,4843,4.843000e+03,4843.00000,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.000000
unique,NaN,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,NaN,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,NaN,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,2421.000000,NaN,1.409628e+05,128.98823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.214536
std,1398.198007,NaN,6.019674e+04,38.99336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.568268
min,0.000000,NaN,-6.400000e+01,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,1210.500000,NaN,1.029135e+05,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,2421.000000,NaN,1.410800e+05,120.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,3631.500000,NaN,1.751955e+05,135.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000


In [ ]:
df.isna().sum()

Unnamed: 0                   0
model_key                    0
mileage                      0
engine_power                 0
fuel                         0
paint_color                  0
car_type                     0
private_parking_available    0
has_gps                      0
has_air_conditioning         0
automatic_car                0
has_getaround_connect        0
has_speed_regulator          0
winter_tires                 0
rental_price_per_day         0
dtype: int64

In [ ]:
df.dtypes

Unnamed: 0                    int64
model_key                    object
mileage                       int64
engine_power                  int64
fuel                         object
paint_color                  object
car_type                     object
private_parking_available      bool
has_gps                        bool
has_air_conditioning           bool
automatic_car                  bool
has_getaround_connect          bool
has_speed_regulator            bool
winter_tires                   bool
rental_price_per_day          int64
dtype: object

# 3) Preprocessing

## 3.a) features and labels

In [ ]:
for col in df.select_dtypes('bool'):
    df[col] = df[col].apply(lambda x: 1 if x else 0)

In [ ]:
df.head(3)

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,1,1,0,0,1,1,1,106
1,1,Citroën,13929,317,petrol,grey,convertible,1,1,0,0,0,1,1,264
2,2,Citroën,183297,120,diesel,white,convertible,0,0,0,0,1,0,1,101


In [ ]:
df.dtypes

Unnamed: 0                    int64
model_key                    object
mileage                       int64
engine_power                  int64
fuel                         object
paint_color                  object
car_type                     object
private_parking_available     int64
has_gps                       int64
has_air_conditioning          int64
automatic_car                 int64
has_getaround_connect         int64
has_speed_regulator           int64
winter_tires                  int64
rental_price_per_day          int64
dtype: object

I recieved an error while transforming becasue there are models which have only one record, hard to split them, we will replace these cars with the categories others, or we will treat them within the APIS becasue we have only three singlr cars which makes it meaningless to predict these three lines of records

In [ ]:
df['model_key'].value_counts()

Citroën        969
Renault        916
BMW            827
Peugeot        642
Audi           526
Nissan         275
Mitsubishi     231
Mercedes        97
Volkswagen      65
Toyota          53
SEAT            46
Subaru          44
Opel            33
Ferrari         33
PGO             33
Maserati        18
Suzuki           8
Porsche          6
Ford             5
KIA Motors       3
Alfa Romeo       3
Fiat             2
Lexus            2
Lamborghini      2
Mini             1
Mazda            1
Honda            1
Yamaha           1
Name: model_key, dtype: int64

In [ ]:
common_brands = ['Citroën', 'Renault', 'BMW', 'Peugeot', 'Audi', 'Nissan', 'Mitsubishi', 'Mercedes', 'Volkswagen', 'Toyota', 'Ferrari', 'Porsche',
                 'Maserati','Suzuki','Porsche','Ford','KIA Motors','Alfa Romeo']

df['model_key'] = df['model_key'].apply(lambda x: x if x in common_brands else 'others')

In [ ]:
df['model_key'].value_counts()

Citroën       969
Renault       916
BMW           827
Peugeot       642
Audi          526
Nissan        275
Mitsubishi    231
others        166
Mercedes       97
Volkswagen     65
Toyota         53
Ferrari        33
Maserati       18
Suzuki          8
Porsche         6
Ford            5
KIA Motors      3
Alfa Romeo      3
Name: model_key, dtype: int64

## replace column name from model_key to name to avoid warning at API docker

In [ ]:
df.rename(columns = {'model_key':'name'}, inplace = True)

In [ ]:
X = df.drop(['Unnamed: 0','rental_price_per_day'], axis=1)
Y = df.loc[:,'rental_price_per_day']

In [ ]:
X.head(2)

,name,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires
0,Citroën,140411,100,diesel,black,convertible,1,1,0,0,1,1,1
1,Citroën,13929,317,petrol,grey,convertible,1,1,0,0,0,1,1


In [ ]:
Y.head(2)

0    106
1    264
Name: rental_price_per_day, dtype: int64

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


# 3.b) pipeline Preprocessing

In [ ]:
numeric_features = ['mileage','engine_power']

# Actually there are neither missing data and no need to scale if it is a decsion tree to be used
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])



categorical_features = ['name','fuel','paint_color', 'car_type',
                        'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
                        'has_speed_regulator', 'winter_tires']

categorical_transformer = OneHotEncoder(drop='first') # no missing values in categorical data, so we only need the OHE


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])



Since this is a regression model and the result is a number no need for the label encoder because it is not a classification case

In [ ]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) # NO  fit in here
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax becaus

Performing preprocessings on train set...
         name  mileage  engine_power    fuel paint_color car_type  \
4550      BMW   132485           135  diesel       white      suv   
1237  Citroën   131121           135  diesel       black   estate   
3158  Renault   209216           135  diesel        grey    sedan   
900   Peugeot   148986           100  diesel       black   estate   
933   Citroën   170500           135  diesel       black   estate   

      private_parking_available  has_gps  has_air_conditioning  automatic_car  \
4550                          1        1                     0              0   
1237                          0        1                     0              0   
3158                          1        1                     0              0   
900                           1        1                     0              0   
933                           1        1                     0              0   

      has_getaround_connect  has_speed_regulator  winter

# 4) model

## Random Forest

In [ ]:
# attention it is a regression situation, use random forest regresion
# RANDOMFOREST
regressor = RandomForestRegressor()

# Grid of values to be tested
params = {
    'max_depth': [3, 8, 10],
    'min_samples_split': [2,4, 8],
    'n_estimators': [20]
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 3, verbose = 2)
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation accuracy : ", gridsearch.best_score_)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END ..max_depth=3, min_samples_split=2, n_estimators=20; total time=   0.1s
[CV] END ..max_depth=3, min_samples_split=2, n_estimators=20; total time=   0.1s
[CV] END ..max_depth=3, min_samples_split=2, n_estimators=20; total time=   0.1s
[CV] END ..max_depth=3, min_samples_split=4, n_estimators=20; total time=   0.1s
[CV] END ..max_depth=3, min_samples_split=4, n_estimators=20; total time=   0.1s
[CV] END ..max_depth=3, min_samples_split=4, n_estimators=20; total time=   0.1s
[CV] END ..max_depth=3, min_samples_split=8, n_estimators=20; total time=   0.1s
[CV] END ..max_depth=3, min_samples_split=8, n_estimators=20; total time=   0.1s
[CV] END ..max_depth=3, min_samples_split=8, n_estimators=20; total time=   0.1s
[CV] END ..max_depth=8, min_samples_split=2, n_estimators=20; total time=   0.4s
[CV] END ..max_depth=8, min_samples_split=2, n_estimators=20; total time=   0.4s
[CV] END ..max_depth=8, min_samples_split=2, n_es

In [ ]:
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))

R2 score on training set :  0.8832941658667977
R2 score on test set :  0.7527544144703197


## XG Boost

In [ ]:
#XGBoost
# attention it is a regression situation, use random forest regresion

regressor = GradientBoostingRegressor()

# Grid of values to be tested
params = {
    'n_estimators': [100, 250],
        'learning_rate': [0.05, 0.1],
        'max_depth': [4, 8],
        'subsample': [0.8],
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 3, verbose = 2)
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation accuracy : ", gridsearch.best_score_)

print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END learning_rate=0.05, max_depth=4, n_estimators=100, subsample=0.8; total time=   0.5s
[CV] END learning_rate=0.05, max_depth=4, n_estimators=100, subsample=0.8; total time=   0.5s
[CV] END learning_rate=0.05, max_depth=4, n_estimators=100, subsample=0.8; total time=   0.5s
[CV] END learning_rate=0.05, max_depth=4, n_estimators=250, subsample=0.8; total time=   1.3s
[CV] END learning_rate=0.05, max_depth=4, n_estimators=250, subsample=0.8; total time=   1.7s
[CV] END learning_rate=0.05, max_depth=4, n_estimators=250, subsample=0.8; total time=   1.5s
[CV] END learning_rate=0.05, max_depth=8, n_estimators=100, subsample=0.8; total time=   1.5s
[CV] END learning_rate=0.05, max_depth=8, n_estimators=100, subsample=0.8; total time=   1.5s
[CV] END learning_rate=0.05, max_depth=8, n_estimators=100, subsample=0.8; total time=   1.4s
[CV] END learning_rate=0.05, max_depth=8, n_estimators=250, subsample=0.8; total time=   3.2s


## Ridge grissearch

In [ ]:
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0,1.5,2] # 0 corresponds to no regularization
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))

...Done.
Best hyperparameters :  {'alpha': 2}
Best R2 score :  0.6961122011226285
R2 score on training set :  0.7077908859467673
R2 score on test set :  0.6802540329565605


In [ ]:
# save the scaler
joblib.dump(preprocessor,'scaler_v3.joblib')


['scaler_v3.joblib']

In [ ]:
# save the model
joblib.dump(gridsearch, 'regressor_ridge_v3.joblib')

['regressor_ridge_v3.joblib']

# Prediction Test

In [ ]:
model = joblib.load('regressor_ridge_v3.joblib')
scaler = joblib.load('scaler_v3.joblib')


In [ ]:
entity ={
  "name": "Peugeot",
  "mileage": 123886,
  "engine_power": 125,
  "fuel": "petrol",
  "paint_color": "black",
  "car_type": "convertible",
  "private_parking_available": 1,
  "has_gps": 0,
  "has_air_conditioning": 0,
  "automatic_car": 0,
  "has_getaround_connect": 0,
  "has_speed_regulator": 1,
  "winter_tires": 1
}

In [ ]:
scaled_X = scaler.transform(entity)

In [ ]:
y_pred = model.predict(scaled_X)

In [ ]:
y_pred

array([123.95873578])

# Save the model

In [ ]:
# to save the model
model = gridsearch
pickle.dump(model, open('model.pkl', 'wb'))

In [ ]:
pickled_model = pickle.load(open('model.pkl', 'rb'))

In [ ]:
pip show scikit-learn


Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: fastai, imbalanced-learn, librosa, lightgbm, mlxtend, qudida, sklearn-pandas, yellowbrick
